# Cirkulære barplot-funktion

Følgende python notebook indeholder funktion til at lave cirkulære barplots af enkelte observatinoer i datasæt over forskeraktiviteter. 

En notebook er opdelt i celler. Celler kan enten være tekstceller (som denne) eller kodeceller (med python kode). Kodeceller kan køres enkeltvis.

For at danne et plot, skal kodecellerne i denne notebook køres i rækkefølge. Den sidste celle indeholder koden, der genererer et plot af en valgt observation. Denne kan derfor køres så mange gange, som man har brug for. 

Flere steder i notebooken er der kodeceller, som kan tilpasses, hvis man vil lave ændringer i plottet (fx søjlelabels og farver). 

Notebooken indeholder følgende:
1. Installation af nødvendige Python pakker
2. Indlæsning af Python pakker
3. Indlæsning af data
4. Indstilling af søjlelabels og søjlefarver
5. Kode til at lave funktionen, der gør det muligt, at lave cirkulært barplot
6. Kørsel af funktion til at lave cirkulært barplot af én observation

Spørgsmål til notebooken skal rettes til CALDISS (caldiss@adm.aau.dk).

### Brug af notebooken

Det antages her, at notebooken er åbnet i JupyterLab. For at køre kodecellerne, placerer man sin markør i cellen og trykker derefter på play-knappen i bjælken over notebooken. Alternativt kan man trykke shift+enter (windows) eller cmd+enter (mac).

Feltet med de kantede parenteser (`[ ]`) til venstre for en celle angiver, om cellen er kørt. Er feltet tomt, er cellen ikke kørt endnu. Indeholder feltet en stjerne (`[*]`), kører koden stadig. Indeholder feltet et tal (`[1]`), er cellen kørt, og man kan fortsætte til næste celle. Tallet angiver kørselsrækkefølgen.

Så længe notebooken er åben, så "gemmes" de ting, som er kørt i notebooken. Lukkes notebooken, så skal alle celler køres igen for at kunne bruge plotting-funktionen (med undtagelse af 1. Installation).

## 1. Installation af nødvendige Python pakker

Cellen herunder installerer de nødvendige Python pakker for at kunne køre kodecellerne. Denne behøves kun køres første gang, at man bruger notebooken.

In [ ]:
!pip install pandas matplotlib openpyxl

## 2. Indlæsning af Python pakker

Nedenstående indlæser de nødvendige python for at kunne køre kodecellerne. Skal køres hver gang, at man åbner notebooken på ny.

In [ ]:
import os
from os.path import join
import json
import pandas as pd
import openpyxl
import random
import re
import numpy as np
import matplotlib.pyplot as plt

from matplotlib.lines import Line2D
from matplotlib import font_manager

## 3. Indlæsning af data

Nedenståedne kodecelle indlæser datasættet, som plots skal genereres ud fra. Data skal angives som en filsti i python variablen `data_path`. 

Det er lettest, hvis notebooken ligger i samme mappe som datasættet. På den måde kan man nøjes med bare at angive navnet på datasættet (husk at filendelsen skal med .xlsx)

In [ ]:
data_path = "./data/simdata_sunburst.xlsx" # <-- Indsæt korrekt sti til datasættet

Når filstien til data er angivet i ovenstående celle, skal de nedenstående celler bare køre, som de er.

In [ ]:
# Indlæs data fra stien
df = pd.read_excel(data_path)

cols_use = [
    'ID - personer',
    'PART_ACAD_norm',
    'PART_NON_norm',
    'PART_INT_norm',
    'PART_ORG_norm',
    'PART_TEACH_norm',
    'PART_QUAL_norm',
    'PART_MEDIA_norm',
    'PROD_MEDIA_norm',
    'PROD_AEST_norm',
    'PROD_ACAD_norm',
    'PROD_POLI_norm',
    'PROD_COMM_norm',
    'INFLOW_norm'
]

# Omdanner datasættet til lang format
df = df[cols_use]
df.loc[:, cols_use[1:]] = df.loc[:, cols_use[1:]] * 100
df_melted = pd.melt(df, id_vars=['ID - personer'], var_name='Level', value_name='Values') # konverterer til langt format, ændrer variabler to "Level" og gemmer værdier fra kolonnerne i "Values" kolonnen

## 4. Indstilling af søjlelabels og søjlefarver

Nedenstående celler bruges til at ændre hhv. søjelabels og søjlefarver. Som standard, beholdes de oprindelige kolonnenavne som søjelabels. Søjler er farvelagt blå, rød og grøn for kolonner, der starter med hhv. "PART_", "PROD_" og "INFLOW_".

### Indstilling af søjlelabels

Ret i koden i nedenstående celle for at ændre søjlelabels. De ønskede labels sættes til højre for `:`. Husk at angive labels i anførselstegn (`""`). Kommaerne skal blive i koden. Hvis label for kolonnen skal udelades for grafen, sæt tom tekst felt "" ind.

In [ ]:
col_labels = {
    "PART_ACAD_norm": "PART_ACAD_norm", # <-- skriv ønsket kolonnelabel til højre for kolon (:). Anførselstegn og komma skal ikke fjernes. 
    "PART_NON_norm": "PART_NON_norm",
    "PART_INT_norm ": "PART_INT_norm ",
    "PART_ORG_norm": "PART_ORG_norm",
    "PART_TEACH_norm": "PART_TEACH_norm",
    "PART_QUAL_norm": "PART_QUAL_norm",
    "PART_MEDIA_norm ": "PART_MEDIA_norm ",
    "PROD_MEDIA_norm": "PROD_MEDIA_norm",
    "PROD_AEST_norm": "PROD_AEST_norm",
    "PROD_ACAD_norm": "PROD_ACAD_norm",
    "PROD_POLI_norm ": "PROD_POLI_norm ",
    "PROD_COMM_norm": "PROD_COMM_norm",
    "INFLOW_norm": "INFLOW_norm",
}

### Indstilling af søjefarver

Ret i koden i nedenstående celle for at ændre søjlefarver. De ønskede farver angives som hex-værdier til højre for `:`. Husk at angive hex-værdi i anførselstegn (`""`). Kommaerne skal blive i koden.

Som standard, bruges blå til PART_-kolonner, rød til PROD_-kolonner og grøn til INFLOW-kolonner.

In [ ]:
chart_colors = {
    "PART_": "#1f77b4", # <-- Skriv ønsket farve. Skal angives som hex-værdi. Anførselstegn og komma skal ikke fjernes. # Blå
    "PROD_": "#d62728", # Rød
    "INFLOW": "#2ca02c" # Grøm
    } 

## 5. Kode til at lave funktionen, der gør det muligt, at lave cirkulært barplot

Den lange kodecelle herunder laver selve funktionen. *Denne skal der ikke foretages ændringer i.*

In [ ]:
def harry_plotter(unique_id, fileout = None, include_values = True, order = None, df_melted = df_melted, chart_colors = chart_colors, col_labels = col_labels):

    df_use = df_melted.copy()

    colour_map = {}

    for col in df_use['Level'].unique():

        if str(col).startswith('PART_'):
            color = chart_colors.get('PART_')
        elif str(col).startswith('PROD_'):
            color = chart_colors.get('PROD_')
        elif str(col).startswith('INFLOW'):
            color = chart_colors.get('INFLOW')
        else:
            continue
        
        colour_map[str(col)] = color

    df_use['colors'] = df_use['Level'].map(colour_map) # Forbinder farver til datasættet


    # filtrer datasættet
    filtered_df = df_use[df_use['ID - personer'] == unique_id]

    # Bruger bestemt definition af strålernes rækkefølge
    # Således strålerne tegnes ud fra datasætetets orden. Listen defineres i nedenstående celle.
    if order is not None:
        new_order = order[3:] + order[:3]
        order_use = [new_order[0]] + list(reversed(new_order[1:]))
        filtered_df['Level'] = pd.Categorical(filtered_df['Level'], order_use)
        filtered_df = filtered_df.sort_values('Level')

    # Plot størrelse og initiering af plot
    plt.figure(figsize=(20,10)) # <-- figure størrelse
    ax = plt.subplot(111, polar=True) # <-- Definition af figur type
    ax.grid(True, linestyle='--', color='gray', alpha=0.6) # Definer grænserne for plottet. Værdier kan ændres. linestyle = type of line for outer circle. color = colour of line. alpha = transparancy 
    
    #Laoyt for akserne
    plt.xticks([]) # Fjerner x-akse labels

    plt.yticks([100, 200], labels =['100%', '200%']) # Definerer labels for y-aksen
    
    #Fjerner de ydre ringe cirkleakserne, som giver mere plads til labels og et renerer udseende
    ax.spines["polar"].set_color("none") 
    ax.spines["start"].set_color("none")
    

    # Constants = parametrer der kontrollerer plot layout:
    upperLimit = 350
    lowerLimit = 0
    labelPadding = 6

    # Beregn max og min for datasættet
    max = filtered_df['Values'].max()

    # Beregning af højde for plottet og input værdier
    slope = (max - lowerLimit) / max
    heights = slope * filtered_df.Values + lowerLimit

    # Beregn strålernes bredde. Total = 2*Pi = 360°
    width = 2*np.pi / 28 #<-- bestemmer bredden af strålerne. Tallet kan ændres

    # Beregn vinkler hvor strålerne er centreret omkring
    indexes = list(range(1, len(filtered_df.index)+1)) # bestem index af dataframe som bruges til vinkel beregning
    angles = np.linspace(0.05, 2 * np.pi - 0.05, len(filtered_df), endpoint=False) # Definer vinkle og position af strålerne

    # Tegn stråler
    # Definer strålernes størrelse, farve og andre layout ting fra tidligere objekter i funktionen
    bars = ax.bar(
        x=angles, 
        height=heights, 
        width=width, 
        bottom=lowerLimit,
        linewidth=1, 
        edgecolor="white",
        color=filtered_df['colors']
    )

    # Nanvgiv labels
    filtered_df['Level'] = filtered_df['Level'].replace(col_labels)

    # Funktion til at tilføje labels
    for bar, angle, height, label, value in zip(bars, angles, heights, filtered_df['Level'], filtered_df["Values"]):

        # Labels rotereres. Rotationen defineres i grader
        rotation = np.rad2deg(angle)

        # Vender labels for bedre læsbarhed
        alignment = ""
        if angle >= np.pi/2 and angle < 3*np.pi/2:
            alignment = "right"
            rotation = rotation + 180
        else: 
            alignment = "left"

        # Definer hvad bar labels indeholder
        if include_values:
            bar_label = f'{label}: {int(round(value, 0))}%'
        else:
            bar_label = f'{label}'

        # Tilføjer labels og deres udseende
        ax.text(
            x=angle, 
            y=lowerLimit + bar.get_height() + labelPadding, 
            s=bar_label, # string der definerer label output
            size = 8.5, #Tekst størrelse. Kan ændres hvis tekst skal være større
            ha=alignment, 
            va='center', # placering af labels
            rotation=rotation, # Rotation af strålerne
            rotation_mode="anchor") # Definerer hvor label skal hænge på strålerne

    if any([fileout != "", fileout is None]):
        if not fileout.endswith(".png"):
            fileout = fileout + ".png"

        plt.savefig(fileout, bbox_inches = 'tight') # Gemmer plottet i lokal mappe

## 6. Kørsel af funktion til at lave cirkulært barplot af én observation

Til sidst skal der sættes de overordnede indstillinger for plottet. Dette gøres ved at tilrette værdierne i cellen herunder. Der skal altid kun rettes i værdierne til venstre for =. 

- `unique_id`: Insæt den korrekte værdi for id. Hvis ID er tekst, så sæt "" rundt om.
- `fileout`: Indsæt navn på plot, som det skal gemmes som. Gemmes ikke, hvis det står tomt. Tom værdi angives ved at sætte "".
- `include_values`: Angiv hvorvidt værdier skal med på søjlerne. True for "Ja", False for Nej (bemærk at hhv. True/False skal skrives med stort forbogstav).
- `order`: Angiv rækkefølgen, som søjler skal stå i (start med kl 12). Rækkefølgen skal angives som en liste af de oprindelige kolonnenavne. Anførselstegn, kommaer og andre specialtegn skal derfor blive i koden. For at undgå slåfejl, kan kolonnenavne evt. flyttes rundt ved at klippe og sætte ind.


In [ ]:
unique_id = 6 # <-- Insæt den korrekte værdi for id. "" <- som string værdi eller tal hvis der er tale om et nummeret id

fileout = "" # <-- indsæt navn på plot, som det skal gemmes som (gemmes ikke, hvis det står tomt)

include_values = True # <-- angiv hvorvidt værdier skal med på søjlerne. True for "Ja", False for Nej.

# Rækkefølge for input til plottet angivet i en liste.
order = [
    "INFLOW_norm",
    "PART_ACAD_norm", 
    "PART_NON_norm", 
    "PART_INT_norm",
    "PART_ORG_norm", 
    "PART_TEACH_norm", 
    "PART_QUAL_norm",
    "PART_MEDIA_norm", 
    "PROD_COMM_norm",
    "PROD_MEDIA_norm", 
    "PROD_AEST_norm",
    "PROD_ACAD_norm", 
    "PROD_POLI_norm"
    ]

Når ovenstående er indstillet, genereres, printes og gemmes (hvis dette er valgt) plottet ved at køre nedenstående celle.

Der gives en advarsel med rød skrift, men denne kan ses bort fra.

In [ ]:
# benytter funktionen og indsættelse af valgte værdier til plottet
circ_barplot = harry_plotter(unique_id, fileout=fileout, include_values=include_values, order = order)

print(circ_barplot) #printer plottet